##  If you are checking the code using Google Colab, check traverse through Table of Contents

# Mandatory step Libraries and Declaration

In [ ]:
import datetime

In [1]:
import json
import requests as rq
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
from google.colab import files
from tqdm import tqdm_notebook

## NLTK
import nltk.data
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

##GenSim
import gensim
import gensim.corpora as corpora
from gensim import corpora
from collections import defaultdict
from gensim import models
from gensim import similarities

#Few Words
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

# Retrieval of Data from RAPIDAPI

In [2]:
'''
Here I am using the RAPIDAPI with ContextualWebSearch News Search.
passing the two keywords seperately and since the RAPIDAPI in Free version only provides 50 articles per request
I have iterated it such way that it gonna fetch 50 articles on every decrement of 6 months span from 15th Oct 2019.
stored the json using the request library in a List
Latestly on 15th Oct , i observed 
'''
url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"
headers = {'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",'x-rapidapi-key': "28876ceca4msh44c02f8c0a33571p1bbfacjsndcdb422e3849" }
querystring = {"fromPublishedDate":"5/1/2015","toPublishedDate":"5/11/2019","autoCorrect":"true","pageNumber":"1","pageSize":"50", "q":"artificial intelligence","safeSearch":"false"}
delta = timedelta(days=183)
listdict = []
keyword = ['artificial intelligence','machine learning']
k = 0
for i in keyword:
    start_date = date(2019, 5, 12)
    end_date = date(2019, 11, 13)
    while start_date > date(1980,1,1):
        try :
          k +=1
          print(k)
          querystring['fromPublishedDate'] = start_date.strftime("%d/%m/%Y")
          querystring['toPublishedDate'] = end_date.strftime("%d/%m/%Y")
          querystring['q']=i
          #print (querystring)
          end_date   -= delta
          start_date -= delta
          response = rq.request("GET", url, headers=headers, params=querystring)
          listdict.extend(response.json()['value'])
        except:
          pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158


# Basic Operation of storing the List of articles in DataFrame and CSV

In [0]:
# Transformed the got List of Dictionary to  pandas DataFrame
Newsdf = pd.DataFrame(listdict)

In [0]:
# Date has been changed the datatype into Datetime64
Newsdf['datePublished'] = Newsdf['datePublished'].astype('datetime64')

In [0]:
#Remove few labels as we got some unneccessary data from JSON Request
Newsdf.drop(labels=['image','isSafe','language','provider'],axis=1,inplace=True)

In [0]:
'''
(execution time takes time)
Extracted the Article content only paragraph. So i have used BeautifulSoup which actually formats the HTML tag
properly. So that you can easily find the content as per the tags and i have added that into the DataFrame
'''
Newsdf['ArticleContent'] = ''
for i,row in Newsdf.iterrows():
  try:
    ArticleContent = ''
    time.sleep(0.01)
    Article = BeautifulSoup(rq.get(Newsdf.at[i,'url'],timeout=15).text)
    Articletext = [i.text.strip() for i in Article.find_all('p')]
    ArticleContent = ' '.join(Articletext[i] for i in range(len(Articletext)))
    Newsdf.at[i,'ArticleContent'] = ArticleContent
  except:
    Newsdf.at[i,'ArticleContent'] = ArticleContent

In [0]:
#Based on ArticleContent , I have created a column based on the content whether the specific word is available or not.
Newsdf['ContentContains'] = ''
for p,row in Newsdf.iterrows():
  if 'artificial intelligence' in Newsdf.ArticleContent.loc[p].lower():
    if 'machine learning' in Newsdf.ArticleContent.loc[p].lower():
      Newsdf.at[p,'ContentContains'] = 'Both'
    else :
      Newsdf.at[p,'ContentContains'] = 'AI'
  elif 'machine learning' in Newsdf.ArticleContent.loc[p].lower():
    Newsdf.at[p,'ContentContains'] = 'ML'
  else :
    Newsdf.at[p,'ContentContains'] = ''

In [0]:
# Delete the Article which doesnt contains either of the word artificial intelligence or machine learning.
Newsdf = Newsdf[Newsdf.ContentContains != '']

In [0]:
#converted the CSV file and download as, there is limit of retrieval of request from RAPIDAPI. 
# So i saved the data.
Newsdf.to_csv('out.csv')
files.download('out.csv')

# Alternate Start point if the RAPIDAPI doesnt work

## make sure you have the out.csv in the present working directory.

In [0]:
#This is alternative start point if my monthly Requests from RAPIDAPI completes.
#for the same reason , i have saved data after Data Cleaning.
try:
    Newsdf.head(1)
except NameError:
    Newsdf = pd.read_csv('out.csv',lineterminator='\n',index_col=0)

In [0]:
# As in the mail , we dont need the URL and removing it now.
# We are also Renaming the columns. As per the Mail and reset the index
try:
  Newsdf.drop(labels=['url','body'],axis=1,inplace=True)
except:
  pass
Newsdf.rename(columns={'title': 'Headline', 'description': 'News_outlet'}, inplace=True)
Newsdf.reset_index(inplace=True)

In [14]:
# Just check if you have article content in the Dataframe.
Newsdf['ArticleContent'][1]

'A survey says more than half feel the federal government is not doing enough to boost the sector, and half have considered leaving B.C. A new survey found that more than half of B.C’s. artificial intelligence companies believe the federal government is not doing enough to boost the sector, and half have considered leaving the province. The non-profit industry association, Artificial Intelligence Network of B.C., says there are more than 150 AI-related firms in B.C. and more than 65 submitted responses to its survey, which was conducted by CityAge and released this week. More than 56 per cent of respondents said the federal government needs to do more to help the local AI sector grow, with 31 per cent saying its efforts were lacking and 24 per cent saying they needed major attention. Half of respondents said they have considered moving their companies out of B.C. They main reasons they gave were a desire to connect to bigger markets (35 per cent) and to operate in a better taxation and

# Articles Preprocessing function

In [0]:
def preprocess_sentence(book):
    ''' Preprocessing invovles 6 steps :
    1) Sentence Tokenizing
    2) Converting to Lower-Case
    3) Word Tokenizing
    4) Word lemmatizing
    5) Removal of Stop Words

    In this function i have also used a trick for seperating the Artificial Intelligence and Machine learning for 
    smoother 
    '''
    cleaned_words = []
    sentences = sent_tokenize(book)
    englishwords = set(nltk.corpus.words.words())
    englishwords.update(['artificial intelligence','machine learning'])
    text = [re.sub('[^\sa-zA-Z]+',r'',text.lower()) for text in sentences]
    text = [i.replace('artificial intelligence','ai',)   for i in text]
    text = [i.replace('machine learning','ml',)   for i in text]
    cleaned_words.extend([word_tokenize(sentence) for sentence in text])
    flatten = lambda outer_list: [item for inner_list in outer_list for item in inner_list]
    words = flatten(cleaned_words)
    words = [re.sub(r'(\bai\b)',r'artificial intelligence',i)    for i in words]
    words = [re.sub(r'(\bml\b)',r'machine learning',i)    for i in words]
    words = [w for w in words if w in englishwords]
    wnl = WordNetLemmatizer()
    lemmatized_words= [wnl.lemmatize(i) for i in words]
    stop_words = set(stopwords.words('english'))
    lemmatized_words = [w for w in lemmatized_words if not w in stop_words]
    return lemmatized_words

In [0]:
# applying the preprocessed article (this takes a bit of time)
texts = [preprocess_sentence(Newsdf['ArticleContent'][i]) for i,row in Newsdf.iterrows()]

In [17]:
#Checking the texts
print(texts)

[['subscribe', 'subscribe', 'st', 'month', 'tax', 'sign', 'star', 'edition', 'change', 'location', 'subscribe', 'subscribe', 'st', 'month', 'tax', 'subscribe', 'subscribe', 'st', 'month', 'tax', 'sign', 'star', 'edition', 'change', 'location', 'copy', 'personal', 'noncommercial', 'use', 'order', 'star', 'content', 'distribution', 'inquire', 'please', 'go', 'trillium', 'automobile', 'association', 'recently', 'annual', 'innovation', 'series', 'auto', 'auto', 'marketing', 'representative', 'spoke', 'artificial intelligence', 'artificial intelligence', 'affecting', 'auto', 'industry', 'could', 'one', 'fundamental', 'technology', 'ahead', 'would', 'agree', 'assessment', 'already', 'artificial intelligence', 'found', 'many', 'useful', 'driving', 'cloud', 'auto', 'insurance', 'driver', 'today', 'alert', 'potentially', 'dangerous', 'warning', 'adaptive', 'cruise', 'control', 'steering', 'among', 'artificial intelligence', 'making', 'efficient', 'operate', 'area', 'autonomous', 'leading', 'for

####Frequency of Corpus

In [18]:
# I wanted to check the words frequency in the documents
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1
print(frequency['machine learning'])

1840


In [0]:
texts = [ [token for token in text if frequency[token] >= 5]    for text in texts]

#GenSim Modelling 

In [0]:
#First created a mapping for words and some index value
dictionary = corpora.Dictionary(texts)
#Below we are taking the Frequency of the each word as per the word's assigned index value above
corpus = [dictionary.doc2bow(text) for text in texts]
# Training LDA Model
lda_model = models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=50,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

## TOPIC-WORD Matrix

In [26]:
lda_model.print_topics()

[(0,
  '0.040*"r" + 0.022*"patient" + 0.021*"health" + 0.020*"cancer" + 0.019*"server" + 0.019*"registration" + 0.018*"medical" + 0.016*"classifier" + 0.015*"tree" + 0.014*"error"'),
 (1,
  '0.029*"cloud" + 0.020*"infrastructure" + 0.019*"security" + 0.016*"mobile" + 0.015*"marketing" + 0.013*"storage" + 0.013*"learning" + 0.012*"edge" + 0.012*"content" + 0.011*"visualization"'),
 (2,
  '0.046*"machine learning" + 0.025*"data" + 0.019*"spark" + 0.013*"apache" + 0.012*"learning" + 0.011*"use" + 0.011*"model" + 0.009*"used" + 0.008*"well" + 0.007*"training"'),
 (3,
  '0.031*"display" + 0.023*"car" + 0.019*"none" + 0.018*"buy" + 0.018*"lake" + 0.017*"solid" + 0.014*"column" + 0.013*"border" + 0.012*"left" + 0.011*"block"'),
 (4,
  '0.127*"data" + 0.047*"machine learning" + 0.021*"big" + 0.017*"business" + 0.015*"analytics" + 0.012*"use" + 0.010*"time" + 0.010*"information" + 0.009*"predictive" + 0.009*"customer"'),
 (5,
  '0.050*"trade" + 0.043*"outcome" + 0.033*"respective" + 0.023*"keyn

## Topic-Document Matrix

In [30]:
term_doc = []
for i in texts:
  bagofwords = dictionary.doc2bow(i)
  term_doc.append(lda_model.get_document_topics(bagofwords,minimum_probability =0.0))
for i in range(len(term_doc)):
  print(term_doc[i])

[(0, 0.00046442106), (1, 0.022740588), (2, 0.0460325), (3, 0.037211414), (4, 0.15316871), (5, 0.00020176254), (6, 0.00024482436), (7, 0.2298635), (8, 0.5090583), (9, 0.0010139237)]
[(0, 0.0004158142), (1, 0.0063356445), (2, 0.010761322), (3, 0.049582135), (4, 0.024307106), (5, 0.00018111586), (6, 0.00021977116), (7, 0.20356657), (8, 0.7037837), (9, 0.00084682705)]
[(0, 0.009038345), (1, 0.025219757), (2, 0.21639946), (3, 0.010429792), (4, 0.045423355), (5, 0.003936895), (6, 0.00477714), (7, 0.09609241), (8, 0.57056713), (9, 0.018115727)]
[(0, 0.00081890007), (1, 0.0024877947), (2, 0.055228993), (3, 0.0009453398), (4, 0.009570199), (5, 0.00035642358), (6, 0.00043249442), (7, 0.4534273), (8, 0.46811926), (9, 0.008613241)]
[(0, 0.00023826153), (1, 0.0006395308), (2, 0.41872263), (3, 0.00027494656), (4, 0.053861596), (5, 0.00010373003), (6, 0.00012586896), (7, 0.4029323), (8, 0.122614466), (9, 0.00048669704)]
[(0, 0.0012734961), (1, 0.02411463), (2, 0.20870343), (3, 0.0014699611), (4, 0.05

##Getting Topics related to this AI and ML

In [31]:
term_AI = lda_model.get_term_topics("artificial intelligence", minimum_probability=0.000001)
term_ML = lda_model.get_term_topics("machine learning", minimum_probability=0.000001)
print(term_AI)
print(term_ML)

[(7, 0.008807539), (8, 0.055085197)]
[(2, 0.04615483), (4, 0.046803262)]


# Thanks for Patience and Contact me if any issue faced.